L'objectif de ce notebook est de donner les clés pour utiliser correctement SubQuery

Application sur l'objet ZoneUrba pour calculer la surface artificielle en provenance de l'OCSGE : failed

Application sur l'objet ZoneUrba pour associer le code insee : in progress

[Doc officielle](https://docs.djangoproject.com/en/4.2/ref/models/expressions/#subquery-expressions)

Exemple :
```python
from django.db.models import OuterRef, Subquery
newest = Comment.objects.filter(post=OuterRef("pk")).order_by("-created_at")
Post.objects.annotate(newest_commenter_email=Subquery(newest.values("email")[:1]))
```

In [1]:
from django.db import connection
from django.db.models import OuterRef, Subquery, Sum, F
from django.contrib.gis.db.models.functions import Area, Intersection, Transform
from django.contrib.gis.db.models import Union

from public_data.models import ZoneUrba, Ocsge, Departement, Commune


def execute(query, params=None):
    with connection.cursor() as cursor:
        cursor.execute(query, params)
        return cursor.fetchall()


ZoneUrba.objects.count(), Ocsge.objects.filter(is_artificial=True).count()

(1010442, 464569)

Test sur une seule ZoneUrba

In [2]:
dept = Departement.objects.get(source_id=32)
zone_urba = ZoneUrba.objects.filter(mpoly__intersects=dept.mpoly).first()
zone_urba

<ZoneUrba: ZoneUrba object (266816)>

In [11]:
surface = (
    Ocsge.objects
    .filter(mpoly__intersects=zone_urba.mpoly)
    .annotate(
        intersection=Intersection("mpoly", zone_urba.mpoly),
        intersection_area=Area(Transform("intersection", 2154)) / 10000,
    )
    .values("year")
    .annotate(artificial_area=Sum("intersection_area"))
    .annotate(artificial_area2=Sum(Area(Transform("mpoly", 2154)) / 10000))
    .order_by("-year")
)

surface

<QuerySet [{'year': 2019, 'artificial_area': 0.3919341124533532, 'artificial_area2': 236.22150755014331}, {'year': 2016, 'artificial_area': 0.391933899382672, 'artificial_area2': 236.22154447013892}]>

In [ ]:
sub_query = (
    Ocsge.objects
    .annotate(geom=OuterRef("mpoly"))
    .filter(mpoly__intersects=F("geom"))
    .annotate(
        intersection=Intersection("mpoly", F("geom")),
        intersection_area=Area(Transform("intersection", 2154)) / 10000,
    )
    .values("year")
    .annotate(artificial_area=Sum("intersection_area"))
    .order_by("-year")
    .values("artificial_area")
)[:1]

In [20]:
sub_query = """
SELECT pdo.year, ST_Area(ST_Transform(ST_Union(pdo.mpoly), 2154)) / 10000 as artificial_area
FROM
    public_data_ocsge pdo
    INNER JOIN public_data_zoneurba pdz ON ST_Intersects(pdo.mpoly, pdz.mpoly) AND pdz.id = 266816
GROUP BY pdo.year
ORDER BY pdo.year DESC
LIMIT 1
"""
with connection.cursor() as cursor:
    cursor.execute(sub_query)
    rows = cursor.fetchall()
rows

[(2019, 236.22150755014363)]

In [24]:
query = f"""
UPDATE public_data_zoneurba
SET artificial_area = subquery.artificial_area
FROM (
    SELECT pdz.id as zone_urba_id, ST_Area(ST_Transform(ST_Union(pdo.mpoly), 2154)) / 10000 as artificial_area
    FROM
        public_data_ocsge pdo INNER JOIN public_data_zoneurba pdz ON ST_Intersects(pdo.mpoly, pdz.mpoly)
    GROUP BY pdz.id
    ORDER BY pdz.id DESC
    LIMIT 10
) AS subquery
WHERE subquery.zone_urba_id = id;
"""
with connection.cursor() as cursor:
    cursor.execute(query)

In [25]:
with connection.cursor() as cursor:
    cursor.execute("SELECT COUNT(*) FROM public_data_zoneurba WHERE artificial_area IS NOT NULL")
    rows = cursor.fetchall()
rows

[(10,)]

In [26]:
query = f"""
UPDATE public_data_zoneurba pdz
SET artificial_area = (
    SELECT ST_Area(ST_Transform(ST_Union(pdo.mpoly), 2154)) / 10000 as artificial_area
    FROM
        public_data_ocsge pdo
    WHERE ST_Intersects(pdo.mpoly, pdz.mpoly)
)
WHERE ;
"""
with connection.cursor() as cursor:
    cursor.execute(query)

InternalError: GEOSIntersects: TopologyException: side location conflict at 0.27068286858737012 43.430616763121137


In [32]:
q = "SELECT COUNT(*) FROM public_data_zoneurba pdz WHERE ST_IsValid(mpoly) IS FALSE AND ST_IsValid(ST_MakeValid(mpoly))"
execute(q)

[(2061,)]

In [34]:
query = """
UPDATE public_data_zoneurba pdz
SET mpoly = ST_Multi(ST_CollectionExtract(ST_MakeValid(mpoly), 3))
WHERE
    ST_IsValid(mpoly) IS FALSE
    AND ST_IsValid(ST_MakeValid(mpoly))
"""
execute(query)

ProgrammingError: no results to fetch

In [ ]:
q = "SELECT COUNT(*) FROM public_data_zoneurba pdz WHERE ST_IsValid(mpoly) IS FALSE"
execute(q)